In [1]:
# Install ultralytics package
%pip install ultralytics -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Disable wandb logging
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [3]:
# Import required libraries
from ultralytics import YOLO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import shutil
from PIL import Image

In [4]:
# Define base directory
base_dir = Path('C://Users//rishi//Desktop//OBJECT_DETECTION_AND_TRACKING//kitti_dataset_yolov8')

In [5]:
# Define image and label paths
img_path = base_dir / 'data_object_image_2/training/image_2'
label_path = base_dir / 'data_object_label_2/training/label_2'

print(img_path)
print(label_path)

# Load class names
with open('C://Users//rishi//Desktop//OBJECT_DETECTION_AND_TRACKING//kitti_dataset_labels//classes.json','r') as f:
    classes = json.load(f)

C:\Users\rishi\Desktop\OBJECT_DETECTION_AND_TRACKING\kitti_dataset_yolov8\data_object_image_2\training\image_2
C:\Users\rishi\Desktop\OBJECT_DETECTION_AND_TRACKING\kitti_dataset_yolov8\data_object_label_2\training\label_2


In [6]:
# Split data into training and validation sets
pairs = list(zip(sorted(list(img_path.glob('*'))), sorted(list(label_path.glob('*')))))
train, test = train_test_split(pairs, test_size=0.1, shuffle=True)


In [7]:
# Create directories for storing train and validation sets
train_path = base_dir / 'train'
train_path.mkdir(exist_ok=True)
valid_path = base_dir / 'valid'
valid_path.mkdir(exist_ok=True)

In [8]:
# Copy images and labels to train and validation directories
for t_img, t_lb in tqdm(train):
    im_path = train_path / t_img.name
    lb_path = train_path / t_lb.name
    shutil.copy(t_img,im_path)
    shutil.copy(t_lb,lb_path)

  0%|          | 0/6732 [00:00<?, ?it/s]

In [9]:
for t_img, t_lb in tqdm(test):
    im_path = valid_path / t_img.name
    lb_path = valid_path / t_lb.name
    shutil.copy(t_img,im_path)
    shutil.copy(t_lb,lb_path)

  0%|          | 0/749 [00:00<?, ?it/s]

In [10]:
# Create kitti.yaml file for defining dataset
yaml_file = 'names:\n'
yaml_file += '\n'.join(f'- {c}' for c in classes)
yaml_file += f'\nnc: {len(classes)}'
yaml_file += f'\ntrain: {str(train_path)}\nval: {str(valid_path)}'
with open('kitti.yaml','w') as f:
    f.write(yaml_file)

In [11]:
# Train YOLOv8n model on kitti dataset
model = YOLO('yolov8n.pt')
train_results = model.train(
    data='C://Users//rishi//Desktop//Main project//kitti.yaml', 
    epochs=10,
    patience=3,
    mixup=0.1,
    project='yolov8n-kitti',
    device=0
)

New https://pypi.org/project/ultralytics/8.1.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.9 🚀 Python-3.11.4 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C://Users//rishi//Desktop//Main project//kitti.yaml, epochs=10, time=None, patience=3, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=yolov8n-kitti, name=train37, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=Non

train: Scanning C:\Users\rishi\Desktop\Main project\train.cache... 7481 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7481/7481 [00:00<?, ?it/s]
val: Scanning C:\Users\rishi\Desktop\Main project\valid.cache... 3051 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3051/3051 [00:00<?, ?it/s]


Plotting labels to yolov8n-kitti\train37\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolov8n-kitti\train37
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      2.49G        1.4      1.888       1.11         39        640: 100%|██████████| 468/468 [02:34<00:00,  3.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 96/96 [00:23<00:00,  4.02it/s]


                   all       3051      16504      0.485      0.361      0.347      0.204

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10       2.5G     0.9635     0.6398     0.9566         43        640: 100%|██████████| 468/468 [02:27<00:00,  3.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 96/96 [00:19<00:00,  4.89it/s]

                   all       3051      16504      0.817      0.632       0.74      0.493



10 epochs completed in 0.829 hours.
Optimizer stripped from yolov8n-kitti\train37\weights\last.pt, 6.2MB
Optimizer stripped from yolov8n-kitti\train37\weights\best.pt, 6.2MB

Validating yolov8n-kitti\train37\weights\best.pt...
Ultralytics YOLOv8.1.9 🚀 Python-3.11.4 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3007208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 96/96 [00:22<00:00,  4.24it/s]


                   all       3051      16504      0.819       0.63      0.741      0.492
                   Car       3051      11609      0.921      0.796      0.909      0.689
            Pedestrian       3051       1892      0.904      0.491      0.672      0.375
                   Van       3051       1201      0.833      0.657      0.779      0.561
               Cyclist       3051        672      0.855      0.518      0.658      0.374
                 Truck       3051        431      0.935      0.833      0.923      0.708
                  Misc       3051        405      0.837      0.506      0.632      0.421
                  Tram       3051        206       0.74      0.777      0.813       0.51
        Person_sitting       3051         88       0.53      0.466      0.541      0.302
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to yolov8n-kitti\train37


In [12]:
# Validate YOLOv8n model on kitti dataset
valid_results = model.val()

Ultralytics YOLOv8.1.9 🚀 Python-3.11.4 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3007208 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\rishi\Desktop\Main project\valid.cache... 3051 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3051/3051 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 191/191 [00:27<00:00,  6.93it/s]


                   all       3051      16504      0.819       0.63       0.74      0.493
                   Car       3051      11609      0.922      0.797      0.909       0.69
            Pedestrian       3051       1892      0.904      0.491       0.67      0.375
                   Van       3051       1201      0.834      0.656      0.779      0.562
               Cyclist       3051        672      0.857      0.517      0.659      0.375
                 Truck       3051        431      0.935      0.833      0.923      0.708
                  Misc       3051        405      0.829      0.506      0.632      0.421
                  Tram       3051        206       0.74      0.777      0.809      0.509
        Person_sitting       3051         88      0.529      0.466       0.54      0.301
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to yolov8n-kitti\train372


In [14]:
# Save trained model
import torch
#torch.save(model.model.state_dict(), 'yolo_model.pt')
# Save the trained model
torch.save(model.state_dict(), 'yolo_model.pt')